In [ ]:
import xarray as xr
import datetime
import pandas as pd
import numpy as np

#first download them:
#wget http://lasp.colorado.edu/data/timed_see/level3/latest_see_L3_merged.ncdf -O SEE_L3_daily.nc
#and then:
nc_file = "/home/ga00693/timed_product/SEE_L3_daily.nc"
ds = xr.open_dataset(nc_file, engine="netcdf4")

# --- Extract wavelengths (center of 1 nm bins) ---
wavelengths = ds["SP_WAVE"].values.squeeze()   # shape: (wavelengths,) → 1.5, 2.5, ..., 190.5, ...

# --- Extract flux (daily spectrum) ---
flux = ds["SP_FLUX"].values.squeeze()          # shape: (time, wavelengths)
flux = np.where(flux == -1.0, np.nan, flux)
dates = ds["DATE"].values.squeeze()            # shape: (time,)


# Convert YYYYDDD to datetime
time = [datetime.datetime.strptime(str(d), "%Y%j") for d in dates]

# Build column names
columns = [f"timed_see_lvl3__{w:.1f}nm__[W/m2/nm]" for w in wavelengths[:-5]]

# Build DataFrame
df = pd.DataFrame(data=flux[18:,:-5], index=time[18:], columns=columns)
df.reset_index(inplace=True)
df.rename(columns={'index':'all__dates_datetime__'}, inplace=True)

#let's forward fill the NaN:
df.ffill(inplace=True)
df.to_csv("timed_see_level3.csv", index=False)